### Переиндексация
Метод reindex - создание нового объекта, данные в котором согласуются с новым инлексом.

In [4]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
obj = Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [15]:
# Если вызвать reindex для этого объекта Series, то данные будут реорганизованы в соответствии с новым индексом,
# а если каких-то из имеющихся в этом индексе значений раньше не было, то вместо них будут подставлены отсутствующие значения:
# new_index = obj.index.sort_values().append(pd.Index(['e']))
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [16]:
# Для упорядоченных данных, например временных рядов, иногда желательно произвести интерполяцию, или восполнение, отсутствующих значений в процессе переиндексации.
# Это позволяет сделать параметр method; так, если задать для него значение ffill, то будет произведено прямое восполнение значений:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [17]:
obj3.reindex(range(6))

0      blue
1       NaN
2    purple
3       NaN
4    yellow
5       NaN
dtype: object

In [18]:
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

In [45]:
# В случае объекта DataFrame функция reindex может изменять строки, столбцы или то и другое.
# Если ей передать просто последовательность, то в результирующем объекте переиндексируются строки:
df_data = np.arange(9).reshape((3, 3))
print(df_data)
frame = pd.DataFrame(
    df_data,
    index=['a', 'c', 'd'],
    columns=['Python', 'PHP', 'Golang'],
)
frame


[[0 1 2]
 [3 4 5]
 [6 7 8]]


,Python,PHP,Golang
a,0,1,2
c,3,4,5
d,6,7,8


In [52]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Python,PHP,Golang
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [51]:
# Столбцы можно переиндексировать, задав ключевое слово columns:
frame.reindex(columns=['Python', 'JS', 'Golang'])

,Python,JS,Golang
a,0,NaN,2
c,3,NaN,5
d,6,NaN,8


In [64]:
frame.loc[['a', 'c'], ['Python', 'Golang']]

,Python,Golang
a,0,2
c,3,5


In [119]:
# fill value определяет какими значениями будет заполнено отсутствующие поля
frame.reindex(index=frame.index.append(pd.Index(['aa', 'bb', 'c'])), fill_value='missing')

,Python,PHP,Golang
a,0,1,2
c,3,4,5
d,6,7,8
aa,missing,missing,missing
bb,missing,missing,missing
c,3,4,5


In [155]:
obj4 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 4, 8])
obj4

0      blue
4    purple
8    yellow
dtype: object

In [159]:
# limit определяет сколько последовательно будет заполнено ячеек при заполнении
obj4.reindex(range(10), method='nearest', limit=1)

0      blue
1      blue
2       NaN
3    purple
4    purple
5    purple
6       NaN
7    yellow
8    yellow
9    yellow
dtype: object

In [165]:
# обратное от limit - мин расстояние между существующими данными и автонаполняемыми
obj4.reindex(range(20), method='ffill', tolerance=1)

0       blue
1       blue
2        NaN
3        NaN
4     purple
5     purple
6        NaN
7        NaN
8     yellow
9     yellow
10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
19       NaN
dtype: object

In [226]:
# levels
report = pd.DataFrame(
    [
        [1, 10, 'John'],
        [1, 20, 'John'],
        [1, 30, 'Tom'],
        [1, 10, 'Bob'],
        [2, 25, 'John'],
        [2, 15, 'Bob'],
    ],
    columns = ['IssueKey', 'TimeSpent', 'User']
)
report

,IssueKey,TimeSpent,User
0,1,10,John
1,1,20,John
2,1,30,Tom
3,1,10,Bob
4,2,25,John
5,2,15,Bob


In [227]:
# сгруппируем по юзерам и задаче, посчитав сумму
time_logged_by_user = report.groupby(['IssueKey', 'User']).TimeSpent.sum()
time_logged_by_user

IssueKey  User
1         Bob     10
          John    30
          Tom     30
2         Bob     15
          John    25
Name: TimeSpent, dtype: int64

In [264]:
# получили составной индекс, можем реиндексировать по какому-либо уровню этого индекса
time_logged_by_user.reindex(index=['Bob', 'Tom'], level='User')

IssueKey  User
1         Bob     10
          Tom     30
2         Bob     15
Name: TimeSpent, dtype: int64

In [313]:
# copy
t_frame = pd.DataFrame(np.random.randn(3, 3), index=['a', 'b', 'c'], columns=['t1', 't2', 't3'])
t_frame

,t1,t2,t3
a,0.725175,-0.430186,0.958405
b,1.689158,1.881049,0.622361
c,1.874729,-0.841233,-1.185806


In [319]:
t_frame2 = t_frame.reindex(index=['a', 'b', 'c'], copy=False)
t_frame2

,t1,t2,t3
a,10.000000,-0.430186,0.958405
b,1.689158,1.881049,0.622361
c,1.874729,-0.841233,-1.185806


In [321]:
# если copy=False то видим что изменения затронули и первоначальный дата фрейм, хотя почему-то t_frame2 is t_frame вернет False
t_frame2.loc['a', 't1'] = 10
t_frame

,t1,t2,t3
a,10.000000,-0.430186,0.958405
b,1.689158,1.881049,0.622361
c,1.874729,-0.841233,-1.185806


In [322]:
# Просто для себя эксперименты
t_frame = pd.DataFrame(np.random.randn(4, 3), index=np.arange(12, step=3), columns=['t1', 't2', 't3'])
t_frame

,t1,t2,t3
0,0.111779,0.403019,0.281159
3,0.412363,-0.158937,-0.606014
6,-0.901376,-0.922520,0.258754
9,0.599479,-0.788158,-0.274803


In [323]:
t_frame.reindex(range(12), method='ffill', tolerance=1)

,t1,t2,t3
0,0.111779,0.403019,0.281159
1,0.111779,0.403019,0.281159
2,NaN,NaN,NaN
3,0.412363,-0.158937,-0.606014
4,0.412363,-0.158937,-0.606014
5,NaN,NaN,NaN
6,-0.901376,-0.922520,0.258754
7,-0.901376,-0.922520,0.258754
8,NaN,NaN,NaN
9,0.599479,-0.788158,-0.274803


### Удаление элементов из оси

In [324]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [327]:
obj['b']

1.0

In [330]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [332]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [334]:
obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [336]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

In [338]:
# Вырезание с помощью меток отличается от обычного вырезания в Python тем, что конечная точка включается:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

In [340]:
# Установка с помощью этих методов модифицирует соответствующий участок Series:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
e    4.0
dtype: float64

In [343]:
# Обращение по индексу к DataFrame предназначено для получения одного или нескольких столбцов путем задания одного значения или последовательности:
data = pd.DataFrame(
    np.arange(16).reshape((4, 4)),
    index=['Python', 'PHP', 'JS', 'Golang'],
    columns=['one', 'two', 'three', 'four']
)
data

,one,two,three,four
Python,0,1,2,3
PHP,4,5,6,7
JS,8,9,10,11
Golang,12,13,14,15


In [345]:
data['two']

Python     1
PHP        5
JS         9
Golang    13
Name: two, dtype: int64

In [354]:
data[['three', 'one']]

,three,one
Python,2,0
PHP,6,4
JS,10,8
Golang,14,12


In [358]:
# У доступа по индексу есть несколько частных случаев. Во-первых, выборка или вырезание данных с помощью булева массива:
data[:2]

,one,two,three,four
Python,0,1,2,3
PHP,4,5,6,7


In [361]:
data[data['three'] > 5]

,one,two,three,four
PHP,4,5,6,7
JS,8,9,10,11
Golang,12,13,14,15


In [364]:
data < 5

,one,two,three,four
Python,True,True,True,True
PHP,True,False,False,False
JS,False,False,False,False
Golang,False,False,False,False


In [367]:
data[data < 5] = 0
data

,one,two,three,four
Python,0,0,0,0
PHP,0,5,6,7
JS,8,9,10,11
Golang,12,13,14,15


### Выборка с помощью loc и iloc

In [369]:
# Для доступа к строкам DataFrame по индексу с помощью меток я ввел специальные индексные операторы loc и iloc.
# Они позволяют выбрать подмножество строк и столбцов DataFrame с применением нотации NumPy, используя либо метки строк (loc), либо целые числа (iloc).
# Выбор одной строки и нескольких столбцов по меткам:
data.loc['Python', ['two', 'three']]

two      0
three    0
Name: Python, dtype: int64

In [373]:
# Произведем аналогичную выборку, но уже по целочисленным индексам:
data.iloc[0, [3, 0, 1]]

four    0
one     0
two     0
Name: Python, dtype: int64

In [375]:
# вторая колонка
data.iloc[2]

one       8
two       9
three    10
four     11
Name: JS, dtype: int64

In [394]:
# с первой по вторую столбец, 3ю, 0вую и 1ю колонки
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
PHP,7,0,5
JS,11,8,9


In [381]:
# Обе функции индексирования работают не только с одиночными метками или списками меток, но и со срезами:
data.loc['PHP':'JS', 'two']

PHP    5
JS     9
Name: two, dtype: int64

In [393]:
# все строки и колонки до 3й-1 где значение 0вой колонки больше 5
data.iloc[:, :2][data.one > 5]

,one,two
JS,8,9
Golang,12,13


In [395]:
# изменим индекс
data.reindex(['PHP', 'Perl'], fill_value='-1')

,one,two,three,four
PHP,0,5,6,7
Perl,-1,-1,-1,-1


In [401]:
# указываем координату ячейки значение которой нужно получить
data._get_value('PHP', 'two')

5

In [403]:
# задаем значение этой ячейки
data._set_value('PHP', 'two', 777)
data

,one,two,three,four
Python,0,0,0,0
PHP,0,777,6,7
JS,8,9,10,11
Golang,12,13,14,15
